In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from bisect import bisect
import operator

In [3]:
def read_data(file_name):
    file_path = org_data_path + file_name
    data = pd.read_csv(file_path,sep=',',names = ['time_counter','accel_base_X','accel_base_Y','accel_base_Z','gyro_base_X',
                                                  'gyro_base_Y','gyro_base_Z','accel_right_X','accel_right_Y','accel_right_Z',
                                                  'gyro_right_X','gyro_right_Y','gyro_right_Z','accel_left_X','accel_left_Y',
                                                  'accel_left_Z','gyro_left_X','gyro_left_Y','gyro_left_Z','fall_detection',
                                                  'fall_check','impact_value'])
    return data

In [9]:
def read_combined_data(file_name,airbag_data):
    file_path = org_data_path + file_name
    data = pd.read_csv(file_path)
    data['Time Point Name'] = pd.to_numeric(data['Time Point Name'], errors='coerce')
    data = data[(data['Time Point Name']>100) & (data['Time Point Name'] < 200) &
                (data['Impact milisec'].notnull()) & ((data['Start milisec']) < (airbag_data.iloc[-1][0]))]
    data = data[['Time Point Name','Start milisec','Impact milisec', 'End Milisec','Duration']]
    return data

In [5]:
# Adding the fall value by choosing the maximum magitude in the fall range from combined data
def add_new_fall_value(airbag_data,manual_data):
    airbag_data['accel_base_mag'] =  np.sqrt(airbag_data['accel_base_X']**2 + airbag_data['accel_base_Y']**2 + 
                                             airbag_data['accel_base_Z']**2)
    for index, row in manual_data.iterrows():
        airbag_data.loc[(airbag_data['time_counter'] >= row['Start milisec']) &
                        (airbag_data['time_counter'] < row['End Milisec']),'fall_check'] = 88
        airbag_data.loc[(airbag_data['accel_base_mag'] == \
                                                  (airbag_data.loc[(airbag_data['time_counter'] >= row['Start milisec']) & \
                                                                   (airbag_data['time_counter'] < row['End Milisec']),
                                                                   'accel_base_mag'].max())) & \
                                                 (airbag_data['time_counter'] >= row['Start milisec']) & \
                                                 (airbag_data['time_counter'] < row['End Milisec']),
                                                 'cal_impact_milisec'] = 80
        if(any(airbag_data['time_counter'] == row['Impact milisec'])):
            airbag_data.loc[(airbag_data['time_counter'] == row['Impact milisec']),'impact_value'] =  84
        else:
            airbag_data.loc[bisect(airbag_data['time_counter'].values, row['Impact milisec']),'impact_value'] =  84

In [56]:
# Three features min acceleration, maximum acceleration, right most acceleration 
# are extracted from the window of 2 seconds range
def min_max_right_feature(data,move_window ,window_time=1000):
    
    move_window += int(window_time/2);
    max_ms = data['time_counter'].max()
    feature = pd.DataFrame(columns={'time_counter','min_acc','max_acc','right_most','fall_value'})
    impact_list = data.loc[(data['cal_impact_milisec'] == 80),'time_counter'].values
    
    end_list = []  #edge condition for non-fall values
    start_list = []
    non_falls_end_list = [0]
    
    time_counter = []
    min_acc = []
    max_acc = []
    right_most = []
    fall_value = []
    
    #prepare window frame 
    for index in impact_list:
        end_list.append(index + int(window_time/2) - move_window)
        start_list.append(index - int(window_time/2) - move_window)
        non_falls_end_list.append(index + 2000)
    start_list.append(max_ms)  #edge condition for non-fall values
    
    #add fall values
    for index in range(len(impact_list)):
        start = start_list[index]
        end = end_list[index]
        
        temp_data = data.loc[(data['time_counter'] >= start) & (data['time_counter'] < end),['time_counter','accel_base_mag']]
        time_counter.append(temp_data.iloc[:,0].min())
        min_acc.append(temp_data.iloc[:,1].min())
        max_acc.append(temp_data.iloc[:,1].max())
        right_most.append(temp_data.tail(1).values[0][1])
        fall_value.append(1)
        
    #add non-fall values
    for start,end in zip(non_falls_end_list,start_list):
        for index in range(start,end,window_time):
            end_index = index + window_time
            if end_index < end:
                temp_data = data.loc[(data['time_counter'] >= index) & (data['time_counter'] < end_index),['time_counter','accel_base_mag']]
                time_counter.append(temp_data.iloc[:,0].min())
                min_acc.append(temp_data.iloc[:,1].min())
                max_acc.append(temp_data.iloc[:,1].max())
                right_most.append(temp_data.tail(1).values[0][1])
                fall_value.append(0)

    feature['time_counter'] = time_counter
    feature['min_acc'] = min_acc
    feature['max_acc'] = max_acc
    feature['right_most'] = right_most
    feature['fall_value'] = fall_value
    return feature

### Folder paths 

In [15]:
org_data_path = "../../../Airbag_Data/"
min_max_path = "../../../Features/min_max_feature/"
min_max_right_path = "../../../Features/min_max_right_feature/"

### Read Data

In [ ]:
# S1 subject 

s1_data=read_data('S1-Airbag.CSV')
s1_combined = read_combined_data('S1-combined.csv',s1_data)
add_new_fall_value(s1_data,s1_combined)

# S2 subject

s2_data=read_data('S2-Airbag.CSV')
s2_combined = read_combined_data('S2-combined.csv',s2_data)
add_new_fall_value(s2_data,s2_combined)

# S3 subject

s3_data=read_data('S3-Airbag.CSV')
s3_combined = read_combined_data('S3-combined.csv',s3_data)
add_new_fall_value(s3_data,s3_combined)

# C1 subject

c1_data=read_data('C1-Airbag.CSV')
c1_combined = read_combined_data('C1-combined.csv',c1_data)
add_new_fall_value(c1_data,c1_combined)

# C2 subject

c2_data=read_data('C2-Airbag.CSV')
c2_combined = read_combined_data('C2-combined.csv',c2_data)
add_new_fall_value(c2_data,c2_combined)

# C3 subject

c3_data=read_data('C3-Airbag.CSV')
c3_combined = read_combined_data('C3-combined.csv',c3_data)
add_new_fall_value(c3_data,c3_combined)

In [22]:
subject_data = [s1_data,s2_data,s3_data,c1_data,c2_data,c3_data]
subjects = ['S1','S2','S3','C1','C2','C3']

## min_max_right acceleration extraction and Saving

In [66]:
%%time
lead_times = [10,100,200,300,400,2000]
for lead_time in lead_times:
    print("extracting features for lead time:{} ms".format(lead_time))
    for i in range(len(subject_data)):
        print("  extracting features for ",subjects[i]," subject")
        feature = min_max_right_feature(subject_data[i],lead_time)
        #saving features of each subject on a particular moving window
        feature.to_csv(min_max_right_path+subjects[i]+"_"+str(lead_time)+ "ms.csv", sep=',',index=False)

extracting features for lead time:10 ms
  extracting features for  S1  subject
  extracting features for  S2  subject
  extracting features for  S3  subject
  extracting features for  C1  subject
  extracting features for  C2  subject
  extracting features for  C3  subject
extracting features for lead time:100 ms
  extracting features for  S1  subject
  extracting features for  S2  subject
  extracting features for  S3  subject
  extracting features for  C1  subject
  extracting features for  C2  subject
  extracting features for  C3  subject
extracting features for lead time:200 ms
  extracting features for  S1  subject
  extracting features for  S2  subject
  extracting features for  S3  subject
  extracting features for  C1  subject
  extracting features for  C2  subject
  extracting features for  C3  subject
extracting features for lead time:300 ms
  extracting features for  S1  subject
  extracting features for  S2  subject
  extracting features for  S3  subject
  extracting featu

In [73]:
s3_data.shape

(1553855, 24)